In [ ]:
!pip install -U weaviate-client

In [ ]:
!pip install Spacy

In [ ]:
import sys
!{sys.executable} -m spacy download en

In [1]:
import spacy
nlp = spacy.load('en_core_web_sm')


In [ ]:
!pip install faker
!pip install nltk
!pip install gensim

In [53]:
import weaviate
import weaviate.classes as wvc
import requests
import json


In [54]:
client = weaviate.connect_to_wcs(
    cluster_url="",
    auth_credentials=weaviate.auth.AuthApiKey("")
    
)


In [55]:
client.is_ready()

True

In [10]:
import weaviate.classes.config as wc
from weaviate.classes.config import Configure, Property, DataType
client.collections.create(
    name="NameSearch",
    properties=[
        wc.Property(name="PersonName", data_type=wc.DataType.TEXT),
        
    ],
    # Define the vectorizer module (none, as we will add our own vectors)
    vectorizer_config=wc.Configure.Vectorizer.none(),
    vector_index_config=Configure.VectorIndex.hnsw(),
)

In [11]:
from faker import Faker  
import spacy
nlp = spacy.load('en_core_web_sm')
import time

t0 = time.time()

fake = Faker()  
DocumentSearch = client.collections.get("NameSearch")
with DocumentSearch.batch.dynamic() as batch:
    for i in range(0, 100000): 
        fakename =fake.name()
        if (i==1):
            print(fakename)  
        properties = {"personName": fakename}
        nlpdoc = nlp(fakename)
        vector=nlpdoc.vector
        batch.add_object(properties,vector=vector)
t1 = time.time()
total = t1-t0
print(total)  

Stacey Brady
575.4583711624146


In [41]:
import weaviate.classes.query as wq
query_text = "Stacey Brady"
import time
doc = nlp(query_text)
query_vector= list(doc.vector)
# Get the collection
SkillSet= client.collections.get("NameSearch")


t0 = time.time()
# Perform query
response = SkillSet.query.near_vector(
    near_vector=query_vector,  # A list of floating point numbers
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    print(
        o.properties["personName"] 
    )  
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )  # P
t1 = time.time()
total = t1-t0
print('Time Taken')
print(total)  

Stacey Brady
Distance to query: -0.000

Stacey Patterson
Distance to query: 0.050

Stacey Garcia
Distance to query: 0.056

Stacey Frank
Distance to query: 0.063

Stacey Nelson
Distance to query: 0.064

Time Taken
0.611290454864502


In [63]:
import weaviate.classes.config as wc
from weaviate.classes.config import Configure, Property, DataType
client.collections.create(
    name="SimpleVectorNameSearch",
    properties=[
        wc.Property(name="PersonName", data_type=wc.DataType.TEXT),
        
    ],
    # Define the vectorizer module (none, as we will add our own vectors)
    vectorizer_config=wc.Configure.Vectorizer.none(),
    vector_index_config=Configure.VectorIndex.hnsw(),
)

In [64]:
def VectorElement(word):
    ListAlphabets = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    ListVectors=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    count = 0
    while count < 26:
        ListVectors[count]= sum(1 for element in word if element.lower()==ListAlphabets[count].lower())
        count =count+1
    return ListVectors

In [65]:
from faker import Faker  
import time

t0 = time.time()

fake = Faker()  
DocumentSearch = client.collections.get("SimpleVectorNameSearch")
with DocumentSearch.batch.dynamic() as batch:
    for i in range(0, 100000): 
        fakename =fake.name()
        if (i==1):
            print(fakename)  
        properties = {"personName": fakename}
        vector=VectorElement(fakename)
        batch.add_object(properties,vector=vector)
t1 = time.time()
total = t1-t0
print(total)  

Anthony Heath
74.02804684638977


In [66]:
import weaviate.classes.query as wq
query_text = "Anthony Heath"
import time
query_vector= VectorElement(query_text)
# Get the collection
SkillSet= client.collections.get("SimpleVectorNameSearch")


t0 = time.time()
# Perform query
response = SkillSet.query.near_vector(
    near_vector=query_vector,  # A list of floating point numbers
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    print(
        o.properties["personName"] 
    )  
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )  # P
t1 = time.time()
total = t1-t0
print('Time Taken')
print(total)  

Anthony Heath
Distance to query: 0.000

Heather Anthony
Distance to query: 0.036

Tony Hahn
Distance to query: 0.057

Anthony Shea
Distance to query: 0.059

Anthony Hahn
Distance to query: 0.061

Time Taken
0.6620504856109619
